<a href="https://colab.research.google.com/github/bdemirc1/llm-smiles/blob/main/Llama_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install autotrain-advanced
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 43.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 72.0 MB/s eta 0

In [18]:
!autotrain setup --update-torch

> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
import pandas as pd

In [7]:
df = pd.read_csv("tox21_balanced_revised_no_id.csv")
df.head(10)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2141 entries, 0 to 2140
Data columns (total 2 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   CCCCCCCC/C=C\CCCCCCCC(N)=O  2141 non-null   object
 1   0                           2141 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 33.6+ KB


In [12]:
text_col = []
for _ , row in df.iterrows():
  input = str(row["CCCCCCCC/C=C\CCCCCCCC(N)=O"])
  response = str(row["0"])
  text = "###Input: " + input + " ###Response: " + response

  text_col.append(text)

df.loc[:, "text"] = text_col
df.head()

,CCCCCCCC/C=C\CCCCCCCC(N)=O,0,text
0,CCCCCCOC(=O)c1ccccc1,0,###Input: CCCCCCOC(=O)c1ccccc1 ###Response: 0
1,O=C(c1ccc(Cl)cc1)c1ccc(Cl)cc1,0,###Input: O=C(c1ccc(Cl)cc1)c1ccc(Cl)cc1 ###Res...
2,COc1cc(Cl)c(OC)cc1N,0,###Input: COc1cc(Cl)c(OC)cc1N ###Response: 0
3,N[C@H](Cc1c[nH]c2ccccc12)C(=O)O,0,###Input: N[C@H](Cc1c[nH]c2ccccc12)C(=O)O ###R...
4,Cc1ccccc1N1C(=O)c2cc(S(N)(=O)=O)c(Cl)cc2NC1C,0,###Input: Cc1ccccc1N1C(=O)c2cc(S(N)(=O)=O)c(Cl...


In [14]:
dt = df.drop(columns=['CCCCCCCC/C=C\CCCCCCCC(N)=O', '0'])
dt.head()

,text
0,###Input: CCCCCCOC(=O)c1ccccc1 ###Response: 0
1,###Input: O=C(c1ccc(Cl)cc1)c1ccc(Cl)cc1 ###Res...
2,###Input: COc1cc(Cl)c(OC)cc1N ###Response: 0
3,###Input: N[C@H](Cc1c[nH]c2ccccc12)C(=O)O ###R...
4,###Input: Cc1ccccc1N1C(=O)c2cc(S(N)(=O)=O)c(Cl...


In [15]:
dt.to_csv("train.csv", index=False)

In [23]:
!autotrain llm \
--train \
--project_name llama-smiles\
--model TinyPixel/Llama-2-7B-bf16-sharded \
--data_path . \
--use_peft \
--use_int4 \
--lr 2e-4 \
--batch-size 1 \
--epochs 2 \
--trainer sft > training.log

> INFO    Running LLM
> INFO    Params: Namespace(version=False, train=True, deploy=False, inference=False, data_path='.', train_split='train', valid_split=None, text_column='text', model='TinyPixel/Llama-2-7B-bf16-sharded', learning_rate=0.0002, num_train_epochs=2, train_batch_size=1, warmup_ratio=0.1, gradient_accumulation_steps=1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, seed=42, add_eos_token=False, block_size=-1, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, project_name='llama-smiles', evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, fp16=False, push_to_hub=False, use_int8=False, model_max_length=1024, repo_id=None, use_int4=True, trainer='sft', target_modules=None, merge_adapter=False, token=None, backend='default', username=None, use_flash_attention_2=False, func=<function run_llm_command_factory at 0x7ca6d0ec9a20>)
> INFO    loading dataset from csv
Loadi